<a href="https://colab.research.google.com/github/neogeolee/Deeplearning/blob/master/Text_movie_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
날짜 : 2020/09/08
이름 : 이태훈
내용 : 영화 리뷰 감성분석하기
"""
import codecs
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

In [ ]:
# 파일 로드 함수정의
def load_data(file):
  result = []

  with open(file, 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()

    for line in lines:
      data = line.split('\t')
      result.append(data)

  result = result[1:] # header정보 제외
  return result

# 데이터셋 생성 함수정의
def make_dataset(train_data, test_data):
  #학습용
  train_x = []
  train_y = []

  for i in range(len(train_data)):
    train_x.append(train_data[i][1])
    train_y.append(int(train_data[i][2]))

  #테스트용
  test_x = []
  test_y = []

  for i in range(len(test_data)):
    test_x.append(test_data[i][1])
    test_y.append(int(test_data[i][2]))

  # 최대 5000개의 단어를 갖는 단어사전 생성
  tokenizer = Tokenizer(num_words=5000)
  tokenizer.fit_on_texts(train_x)

  # BOW 표기
  token_train_x = tokenizer.texts_to_sequences(train_x)
  token_test_x = tokenizer.texts_to_sequences(test_x)

  return (token_train_x, train_y), (token_test_x, test_y)

In [ ]:
# 데이터 파일 로드
train_data = load_data('/content/drive/My Drive/Tensorflow_works/data/ratings_train.txt')
test_data = load_data('/content/drive/My Drive/Tensorflow_works/data/ratings_test.txt')
train_data[0]

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']

In [ ]:
# 데이터셋 생성
(train_x, train_y), (test_x, test_y) = make_dataset(train_data, test_data)

In [ ]:
# 데이터셋 확인
print(train_x[0], train_y[0])
print(train_x[1], train_y[1])
print(test_x[0], test_y[0])
print(test_x[49999], test_y[49999])

[23, 936, 4, 1097] 0
[602] 1
[639, 49] 1
[56, 1376] 0


In [ ]:
# 데이터 전처리
train_x = sequence.pad_sequences(train_x, maxlen=100)
test_x = sequence.pad_sequences(test_x, maxlen=100)

train_y = np.array(train_y)
test_y = np.array(test_y)

train_x[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,   23,  936,    4,
       1097], dtype=int32)

In [ ]:
# 모델 구성
model = Sequential()
model.add(Embedding(5000, 128))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         640000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 771,713
Trainable params: 771,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 설정
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
# 모델 학습 - 1시간 소요
model.fit(train_x, 
          train_y,
          batch_size = 128,
          epochs = 10)

Epoch 1/10
1172/1172 [==============================] - 46s 39ms/step - loss: 0.4493 - acc: 0.7666
Epoch 2/10
1172/1172 [==============================] - 43s 37ms/step - loss: 0.4035 - acc: 0.7924
Epoch 3/10
1172/1172 [==============================] - 42s 36ms/step - loss: 0.3862 - acc: 0.7998
Epoch 4/10
1172/1172 [==============================] - 42s 36ms/step - loss: 0.3714 - acc: 0.8069
Epoch 5/10
1172/1172 [==============================] - 42s 36ms/step - loss: 0.3596 - acc: 0.8132
Epoch 6/10
1172/1172 [==============================] - 41s 35ms/step - loss: 0.3475 - acc: 0.8181
Epoch 7/10
1172/1172 [==============================] - 41s 35ms/step - loss: 0.3334 - acc: 0.8259
Epoch 8/10
1172/1172 [==============================] - 41s 35ms/step - loss: 0.3190 - acc: 0.8332
Epoch 9/10
1172/1172 [==============================] - 41s 35ms/step - loss: 0.3049 - acc: 0.8395
Epoch 10/10
1172/1172 [==============================] - 41s 35ms/step - loss: 0.2913 - acc: 0.8459


In [ ]:
# 모델 성능확인
result = model.evaluate(test_x, test_y, batch_size=128)
result

391/391 [==============================] - 5s 13ms/step - loss: 0.5344 - acc: 0.7670


[0.5343965291976929, 0.766979992389679]

In [ ]:
# 모델 저장
model.save('/content/drive/My Drive/Tensorflow_works/model/review1.model')

INFO:tensorflow:Assets written to: /content/drive/My Drive/Tensorflow_works/model/review1.model/assets
